In [3]:
from utils.df_handle import *

In [1]:
import requests

In [20]:
import requests
import pandas as pd
url = 'https://giaoduocpham.com/report/xlsx/rebase_field_service.stock_delivery_customer_xlsx?options={"form":{"date_from":"2022-07-07","date_to":"2022-07-14","select_date":"7days_ago_to_now","company_id":1,"display_name":"stock.delivery.customer.report,6659","create_uid":412,"write_date":"2022-07-14 10:22:57","id":6659,"customer_ids":[73069],"__last_update":"2022-07-14 10:22:57","create_date":"2022-07-14 10:22:57","write_uid":412},"model":"stock.delivery.customer.report","ids":[]}'
r = requests.post(url)
# open('temp.xls', 'wb').write(r.content)
# df = pd.read_excel('temp.xls', engine='openpyxl')

In [21]:
r.content

b'\n        \n        \n        <!DOCTYPE html>\n        <html lang="vi-VN" data-website-id="1" data-oe-company-name="Logistics Pharma">\n            <head>\n                <meta charset="utf-8"/>\n                <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"/>\n                <meta name="viewport" content="width=device-width, initial-scale=1, user-scalable=no"/>\n\n                <title>Rebase</title>\n        <link type="image/x-icon" rel="shortcut icon" href="/web/image/website/1/favicon/"/>\n        <script type="text/javascript">\n                    var odoo = {\n                        csrf_token: "aff871f739a2f52a7d08c400e575a799417cc6b3o",\n                    };\n                </script>\n\n                \n            <meta name="generator" content="Rebase"/>\n\n            \n            \n            \n            \n            \n            \n                \n                \n                \n                    \n                        <meta prope

In [8]:
r = requests.get("https://giaoduocpham.com/report/xlsx/rebase_field_service.stock_delivery_customer_xlsx?options={%22form%22:{%22date_from%22:%222022-07-07%22,%22date_to%22:%222022-07-14%22,%22select_date%22:%227days_ago_to_now%22,%22company_id%22:1,%22display_name%22:%22stock.delivery.customer.report,6658%22,%22create_uid%22:412,%22write_date%22:%222022-07-14%2010:03:01%22,%22id%22:6658,%22customer_ids%22:[73069],%22__last_update%22:%222022-07-14%2010:03:01%22,%22create_date%22:%222022-07-14%2010:03:01%22,%22write_uid%22:412},%22model%22:%22stock.delivery.customer.report%22,%22ids%22:[]}&context={%22tz%22:%22Asia/Ho_Chi_Minh%22,%22lang%22:%22vi_VN%22,%22uid%22:412,%22xls_export%22:1,%22active_model%22:%22stock.delivery.customer.report%22,%22active_id%22:6658,%22active_ids%22:[6658]}")

In [10]:
# output = open('test.xls', 'wb')
# output.write(r.content)
# output.close()

In [13]:
r.content

b'\n        \n        \n        <!DOCTYPE html>\n        <html lang="vi-VN" data-website-id="1" data-oe-company-name="Logistics Pharma">\n            <head>\n                <meta charset="utf-8"/>\n                <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"/>\n                <meta name="viewport" content="width=device-width, initial-scale=1, user-scalable=no"/>\n\n                <title>Rebase</title>\n        <link type="image/x-icon" rel="shortcut icon" href="/web/image/website/1/favicon/"/>\n        <script type="text/javascript">\n                    var odoo = {\n                        csrf_token: "a6dbaa1412c727e342cf7633f8fdaae53c6b3a3co",\n                    };\n                </script>\n\n                \n            <meta name="generator" content="Rebase"/>\n\n            \n            \n            \n            \n            \n            \n                \n                \n                \n                    \n                        <meta prope

In [12]:
excel_data = pd.read_excel(r.content, engine='openpyxl')

BadZipFile: File is not a zip file

In [11]:
import pandas as pd
import io

file_extension = "xlsx"

# (...)

if file_extension == 'xlsx':
    excel_data = pd.read_excel(r.content, engine='openpyxl')
elif file_extension == 'xls':
    excel_data = pd.read_excel(r.content)

BadZipFile: File is not a zip file

In [ ]:
# from utils.df_handle import *
import pendulum
from airflow import DAG
from airflow.operators.dummy_operator import DummyOperator
from airflow.operators.python_operator import PythonOperator

local_tz = pendulum.timezone("Asia/Bangkok")

name='TICHLUYTRANGOAIQ2'
prefix='KM_'
csv_path = '/usr/local/airflow/plugins'+'/'

dag_params = {
    'owner': 'airflow',
    "depends_on_past": False,
    'start_date': datetime(2022, 5, 10, tzinfo=local_tz),
    'email_on_failure': True,
    'email_on_retry': False,
    'email':['duyvq@merapgroup.com', 'vanquangduy10@gmail.com'],
    'do_xcom_push': False,
    'execution_timeout':timedelta(seconds=300)
    # 'retries': 3,
    # 'retry_delay': timedelta(minutes=10),
}

dag = DAG(prefix+name,
          catchup=False,
          default_args=dag_params,
          schedule_interval= '*/30 6-23 * * *',
          tags=[prefix+name, 'Sync', '30mins']
)

In [4]:
sql1 = \
f"""
SELECT
GETDATE() as Inserted_At,
s.SlsperID,
r.BranchID,
r.SlsperID as RouteID,
r.CustID,
r.AccumulateID,
r.LevelID,
r.Status,
r.Crtd_DateTime
FROM dbo.OM_AccumulatedRegis r
INNER JOIN dbo.OM_SalespersonRoute s ON s.SalesRouteID=r.SlsperID
WHERE AccumulateID='CSBH22Q2-14QD/MR'
"""
sql2 = \
f"""
SELECT
CustID,
AccumulatedValue, 
Reward FROM dbo.OM_AccumulatedResult
WHERE AccumulateID='CSBH22Q2-14QD/MR'
"""
sql3 = \
f"""
SELECT
CustID,
sum(Amt) as PaidAmt 
FROM dbo.OM_SalesOrdAccumulate WHERE AccumulateID='CSBH22Q2-14QD/MR' group by CustID
"""
df1 = get_ms_df(sql1)
df2 = get_ms_df(sql2)
df3 = get_ms_df(sql3)
dfa = df1.merge(df2, on=['CustID'], how='left')
# df.head()
dfa = dfa.merge(df3, on=['CustID'], how='left')
dfa.PaidAmt.fillna(0, inplace=True)

In [5]:
sql1 = \
f"""
SELECT
GETDATE() as Inserted_At,
s.SlsperID,
r.BranchID,
r.SlsperID as RouteID,
r.CustID,
r.AccumulateID,
r.LevelID,
r.Status,
r.Crtd_DateTime
FROM dbo.OM_AccumulatedRegis r
INNER JOIN dbo.OM_SalespersonRoute s ON s.SalesRouteID=r.SlsperID
WHERE AccumulateID='CSBH22Q2-04QD/PN'
"""
sql2 = \
f"""
SELECT
CustID,
AccumulatedValue, 
Reward FROM dbo.OM_AccumulatedResult
WHERE AccumulateID='CSBH22Q2-04QD/PN'
"""
sql3 = \
f"""
SELECT
CustID,
sum(Amt) as PaidAmt 
FROM dbo.OM_SalesOrdAccumulate WHERE AccumulateID='CSBH22Q2-04QD/PN' group by CustID
"""
df1 = get_ms_df(sql1)
df2 = get_ms_df(sql2)
df3 = get_ms_df(sql3)
dfb = df1.merge(df2, on=['CustID'], how='left')
# df.head()
dfb = dfb.merge(df3, on=['CustID'], how='left')
dfb.PaidAmt.fillna(0, inplace=True)

In [6]:
# dfb.shape

In [7]:
df = union_all([dfa,dfb])

In [10]:
bq_values_insert(df, "f_tichluytrangoaiq2", 3)

1it [00:05,  5.24s/it]


In [ ]:
dummy_start = DummyOperator(task_id="dummy_start", dag=dag)

insert = PythonOperator(task_id="insert", python_callable=insert, dag=dag)

update = PythonOperator(task_id="update", python_callable=update, dag=dag)

update_sync_dms_ardoc_2 = PythonOperator(task_id="update_sync_dms_ardoc_2", python_callable=update_sync_dms_ardoc_2, dag=dag)

dummy_start >> update >> insert >> update_sync_dms_ardoc_2